In [1]:
%matplotlib widget

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

from PfyMU.gait.train_classifier.core import load_datasets

In [3]:
gait_sets_path = Path('/Users/adamol/Documents/Datasets/gait/processed')

datasets = [
    gait_sets_path / 'bluesky2',
    gait_sets_path / 'daliac',
    gait_sets_path / 'ltmm',
    gait_sets_path / 'usc-had'
]

X, Y, subjects, activities = load_datasets(datasets, goal_fs=50.0, acc_mag=True, window_length=3.0, window_step=[0.75, 0.25])

In [4]:
from scipy.signal import butter, sosfiltfilt

sos = butter(4, [0.2/25, 10./25], btype='band', output='sos')
X = sosfiltfilt(sos, X, axis=1)

## Dataset class summary

In [5]:
print('Total samples (3.0s windows): ', Y.size)
print('Total walking samples: ', Y.sum())
print('Total non-walking samples: ', Y.size - Y.sum(), '\n')
print(f'% walking samples: {Y.sum() / Y.size * 100:.2f}')

Total samples (3.0s windows):  74926
Total walking samples:  29963
Total non-walking samples:  44963 

% walking samples: 39.99


## Feature Generation

In [6]:
from PfyMU.features import *

In [7]:
FB = Bank(window_length=None, window_step=None)

# add features
FB + Mean()
FB + MeanCrossRate()
FB + StdDev()
FB + Skewness()
FB + Kurtosis()
FB + Range()
FB + IQR()
FB + RMS()
FB + Autocorrelation(lag=1, normalize=True)
FB + LinearSlope()
FB + SignalEntropy()
FB + SampleEntropy(m=4, r=1.0)
FB + PermutationEntropy(order=3, delay=1, normalize=True)
FB + ComplexityInvariantDistance(normalize=True)
FB + RangeCountPercentage(range_min=0, range_max=1.0)
FB + RatioBeyondRSigma(r=2.0)
FB + JerkMetric(normalize=True)
FB + DimensionlessJerk(log=True, signal_type='acceleration')
FB + SPARC()
FB + DominantFrequency(low_cutoff=0.25, high_cutoff=5.0)
FB + DominantFrequencyValue(low_cutoff=0.25, high_cutoff=5.0)
FB + PowerSpectralSum(low_cutoff=0.25, high_cutoff=5.0)
FB + SpectralFlatness(low_cutoff=0.25, high_cutoff=5.0)
FB + SpectralEntropy(low_cutoff=0.25, high_cutoff=5.0)
FB + DetailPower(wavelet='coif4', freq_band=[1.0, 3.0])
FB + DetailPowerRatio(wavelet='coif4', freq_band=[1.0, 3.0])

In [8]:
X_feat = FB.compute(X, fs=50.0, windowed=True)

/Users/adamol/opt/miniconda3/envs/pfymu/lib/python3.8/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 6 is too high: all coefficients will experience boundary effects.
  warnings.warn(


## Feature Exploration

In [9]:
feats = pd.DataFrame(data=X_feat, columns=[i.parent._name for i in FB._feat_list])
feats['Labels'] = Y

feat_names = [i.parent._name for i in FB._feat_list]

### Feature distributions

In [10]:
plt.close('all')

f, ax = plt.subplots(nrows=7, ncols=4, figsize=(10, 10), sharex=True)

j, k = 0, 0
for i, ft in enumerate([i.parent._name for i in FB._feat_list]):
    if k > 3:
        j += 1
        k  = 0
    sns.violinplot(x='Labels', y=ft, data=feats, ax=ax[j, k])
    ax[j, k].set_ylabel(None)
    ax[j, k].set_title(ft)
    ax[j, k].set_xlabel(None)
    
    k += 1

for k in range(4):
    ax[-1, k].set_xticklabels(['NG', 'G'])

f.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Predictive Power Score

In [11]:
import ppscore

In [12]:
df_predictors = ppscore.predictors(feats, 'Labels', output='df')
plt.figure()
ax = sns.barplot(data=df_predictors, x="x", y="ppscore")
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
pps_matrix = ppscore.matrix(feats)

In [14]:
plt.figure(figsize=(15, 10))
sns.heatmap(pps_matrix, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=True)
plt.tight_layout()
# plt.savefig('PPScore_matrix.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
from sklearn.decomposition import PCA

In [16]:
pca = PCA(n_components=5)
x_pca = pca.fit_transform(feats)
df_pca = pd.DataFrame(x_pca, columns=[f'PC {i+1}' for i in range(x_pca.shape[1])])
df_pca['Labels'] = Y
df_pca.head(2)

,PC 1,PC 2,PC 3,PC 4,PC 5,Labels
0,-2509.399045,7.311646,3.403297,2.185882,1.853542,1
1,-2509.349954,6.553833,1.539056,2.706662,2.189520,1


In [17]:
sns.pairplot(df_pca, hue='Labels')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
from separation_stats import db_2class, corr_select, cohen_d, ttest_select, mwu_select, auc_score

In [19]:
sep_df = pd.DataFrame()
sep_df['Feature'] = feat_names

sep_df['DBI'], db_rank = db_2class(X_feat, Y)
sep_df['Correlation r'] = corr_select(X_feat, Y)
sep_df['T-test p'], sep_df['Cohens d'] = ttest_select(X_feat, Y)
sep_df['MWU-test p'], _ = mwu_select(X_feat, Y)
sep_df['AUC'] = auc_score(X_feat, Y)

sep_df['Cohens d'] = sep_df['Cohens d'].abs()
sep_df['AUC'] = sep_df['AUC'].apply(lambda i: 1 - i if i < 0.5 else i)

In [20]:
sep_df.sort_values('DBI')

,Feature,DBI,Correlation r,T-test p,Cohens d,MWU-test p,AUC
1,MeanCrossRate,1.162966,-0.516103,0.000000e+00,1.229993,0.000000e+00,0.814807
23,SpectralEntropy,1.255501,-0.510837,0.000000e+00,1.212979,0.000000e+00,0.808381
13,ComplexityInvariantDistance,1.267872,-0.539467,0.000000e+00,1.307845,0.000000e+00,0.826914
8,Autocorrelation,1.307178,0.532574,0.000000e+00,1.284455,0.000000e+00,0.826905
6,IQR,1.309579,0.411281,0.000000e+00,0.921054,0.000000e+00,0.902851
22,SpectralFlatness,1.316660,-0.521933,0.000000e+00,1.249047,0.000000e+00,0.818776
5,Range,1.317883,0.389035,0.000000e+00,0.862047,0.000000e+00,0.895833
2,StdDev,1.328613,0.380355,0.000000e+00,0.839516,0.000000e+00,0.898837
7,RMS,1.328613,0.380355,0.000000e+00,0.839516,0.000000e+00,0.898837
12,PermutationEntropy,1.492345,-0.546277,0.000000e+00,1.331315,0.000000e+00,0.834840


In [21]:
sep_df.sort_values('Cohens d', ascending=False)

,Feature,DBI,Correlation r,T-test p,Cohens d,MWU-test p,AUC
12,PermutationEntropy,1.492345,-0.546277,0.000000e+00,1.331315,0.000000e+00,0.834840
13,ComplexityInvariantDistance,1.267872,-0.539467,0.000000e+00,1.307845,0.000000e+00,0.826914
8,Autocorrelation,1.307178,0.532574,0.000000e+00,1.284455,0.000000e+00,0.826905
22,SpectralFlatness,1.316660,-0.521933,0.000000e+00,1.249047,0.000000e+00,0.818776
1,MeanCrossRate,1.162966,-0.516103,0.000000e+00,1.229993,0.000000e+00,0.814807
23,SpectralEntropy,1.255501,-0.510837,0.000000e+00,1.212979,0.000000e+00,0.808381
6,IQR,1.309579,0.411281,0.000000e+00,0.921054,0.000000e+00,0.902851
5,Range,1.317883,0.389035,0.000000e+00,0.862047,0.000000e+00,0.895833
7,RMS,1.328613,0.380355,0.000000e+00,0.839516,0.000000e+00,0.898837
2,StdDev,1.328613,0.380355,0.000000e+00,0.839516,0.000000e+00,0.898837


In [22]:
sep_df.sort_values('AUC', ascending=False)

,Feature,DBI,Correlation r,T-test p,Cohens d,MWU-test p,AUC
6,IQR,1.309579,0.411281,0.000000e+00,0.921054,0.000000e+00,0.902851
7,RMS,1.328613,0.380355,0.000000e+00,0.839516,0.000000e+00,0.898837
2,StdDev,1.328613,0.380355,0.000000e+00,0.839516,0.000000e+00,0.898837
5,Range,1.317883,0.389035,0.000000e+00,0.862047,0.000000e+00,0.895833
24,DetailPower,32724.930987,-0.005634,1.230299e-01,0.011501,0.000000e+00,0.856356
12,PermutationEntropy,1.492345,-0.546277,0.000000e+00,1.331315,0.000000e+00,0.834840
13,ComplexityInvariantDistance,1.267872,-0.539467,0.000000e+00,1.307845,0.000000e+00,0.826914
8,Autocorrelation,1.307178,0.532574,0.000000e+00,1.284455,0.000000e+00,0.826905
22,SpectralFlatness,1.316660,-0.521933,0.000000e+00,1.249047,0.000000e+00,0.818776
1,MeanCrossRate,1.162966,-0.516103,0.000000e+00,1.229993,0.000000e+00,0.814807


### Pair plots (Top Features)

In [23]:
top_feats = ['PowerSpectralSum', 'PermutationEntropy', 'IQR', 'DominantFrequency', 'DimensionlessJerk', 'DetailPowerRatio']

sns.pairplot(feats, hue='Labels', vars=top_feats)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …